# Which vectorization?

In [2]:
!pip install mlflow boto3 awscli

In [4]:
import mlflow
import os
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri(os.getenv("MLFLOW_SERVER_TRACKING_URI_EC2"))

In [5]:
# Set or create an experiment
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

2025/07/16 19:11:58 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://youtube-sentimental-bucket/356399685852430345', creation_time=1752673322579, experiment_id='356399685852430345', last_update_time=1752673322579, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [8]:
df = pd.read_csv(r'D:\bybappy\youtube_sentimental_analyzer\notebooks\artifacts\dataset.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
import os
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Ensure artifacts directory exists
os.makedirs("artifacts", exist_ok=True)

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(
        df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
    )

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", str(ngram_range))  # must be string
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log model params
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Train model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Predict
        y_pred = model.predict(X_test)

        # Log metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Confusion Matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        
        # Save to artifacts with unique name
        cm_filename = f"confusion_matrix_{vectorizer_name}_{ngram_range[0]}_{ngram_range[1]}.png"
        cm_path = os.path.join("artifacts", cm_filename)
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

        # Log model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range[0]}_{ngram_range[1]}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]
max_features = 5000

for ngram_range in ngram_ranges:
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/07/16 19:15:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/16 19:16:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-54-196-196-75.compute-1.amazonaws.com:5000/#/experiments/356399685852430345/runs/8f488e4611464997bd6270fe93acbbdc.
2025/07/16 19:16:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-196-196-75.compute-1.amazonaws.com:5000/#/experiments/356399685852430345.
2025/07/16 19:16:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/16 19:17:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-54-196-196-75.compute-1.amazonaws.com:5000/#/experiments/356399685852430345/run